In [ ]:
from langchain_core.tools import tool
from kg_manager_new import KGManager

kg_manager = KGManager(ttl_path="Test_filled.ttl", case_sensitive=False)
kg_manager.load()

@tool
def check_known_failures(skill_name: str, snapshot_json: str):
    """
    Sucht im FMEA_KG nach bekannten FailureModes für einen Skill und vergleicht
    sie mit dem aktuellen Variablen-Snapshot.
    Gibt die beste Übereinstimmung zurück.
    """
    # Hier Ihre Logik: 
    # 1. SPARQL: Finde alle FailureModes die 'preventsFunction' <Skill> haben.
    # 2. Python: Vergleiche 'hasFailureModeParams' JSON mit snapshot_json.
    return "UnknownFailure" 

@tool
def get_plc_context(fb_name: str):
    """
    Holt die Schnittstelle (Inputs/Outputs) und genutzte Variablen eines FBs 
    aus dem Test_filled.ttl, damit korrekter Code generiert wird.
    """
    # Nutzt kg_manager._pou_lookup und gibt den ConsistencyReport zurück
    # oder führt eine gezielte SPARQL Query aus.
    return kg_manager.get_consistency_report_for(fb_name) # (Müsste implementiert werden)

In [ ]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

class AgentState(TypedDict):
    event_payload: dict
    current_hypothesis: str
    generated_code: str
    execution_result: str
    iterations: int

# --- Nodes (Funktionen für die Zustände) ---

def diagnose_node(state: AgentState):
    print("--- DIAGNOSE ---")
    skill = state['event_payload']['lastExecutedSkill']
    snapshot = state['event_payload']['snapshot']
    
    # LLM entscheidet hier, ob Tools aufgerufen werden
    # Prompt: "Analysiere den Snapshot für Skill {skill}. Nutze check_known_failures."
    
    # ... Logik ...
    return {"current_hypothesis": "Verdacht auf Sensorfehler A02"}

def plan_monitoring_node(state: AgentState):
    print("--- PLAN MONITORING ---")
    hypothesis = state['current_hypothesis']
    
    # RAG Zugriff auf Test_filled.ttl um Variablennamen zu finden
    # Prompt: "Erstelle ST-Code für eine Monitoring Action, die Sensor A02 prüft. 
    # Nutze nur Variablen aus dem Kontext von FB_Automatikbetrieb."
    
    code = """
    // Generierter ST Code
    IF inputs[0].v < 10 THEN ...
    """
    return {"generated_code": code}

def execute_node(state: AgentState):
    print("--- EXECUTE ON PLC ---")
    # Hier Code Injection via XML und OPC UA Aufruf
    # result = opcua_client.call_method(...)
    result = "Sensor_Value_Is_0"
    return {"execution_result": result}

def evaluate_node(state: AgentState):
    print("--- EVALUATE ---")
    # LLM bewertet das Ergebnis
    if state['execution_result'] == "Sensor_Value_Is_0":
        return {"decision": "root_cause_found"}
    else:
        return {"decision": "try_again"}

# --- Graph Definition ---

workflow = StateGraph(AgentState)

workflow.add_node("diagnose", diagnose_node)
workflow.add_node("plan_monitoring", plan_monitoring_node)
workflow.add_node("execute", execute_node)
workflow.add_node("evaluate", evaluate_node)

workflow.set_entry_point("diagnose")

workflow.add_edge("diagnose", "plan_monitoring")
workflow.add_edge("plan_monitoring", "execute")
workflow.add_edge("execute", "evaluate")

# Bedingte Kante
def decide_next_step(state):
    if state.get("decision") == "root_cause_found":
        return END # Oder weiter zu SystemReaction
    return "plan_monitoring" # Loop (mit Abbruchbedingung in Realität)

workflow.add_conditional_edges("evaluate", decide_next_step)

app = workflow.compile()